In [1]:
### Initialized a Embedding Model.

from langchain_ollama import OllamaEmbeddings

embed = OllamaEmbeddings(
    model="llama3"
)

In [2]:
input_texts = ["Document 1...", "Document 2..."]
vectors = embed.embed_documents(input_texts)
print(len(vectors))

2


In [3]:
from IPython.display import Markdown, display
import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import StorageContext

### ChromaDB 

In [18]:
# create client and a new collection
chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection(name="orangehrm")


In [30]:
chroma_collection.add(

    documents = [
        """
{
    user_instruction : "What are steps to login",
    System_output : 
                    [
                    Step1 : Click on the Username inputbox
                    Step2 : Enter the Username in the Username inputbox
                    Step3 : click on the password inputbox
                    Step4 : Enter the password in the password inputbox
                    Step5 : Click Login Button
                    ]
}
""",
        """
{
    user_instruction : "What are steps to logout",
    System_output : 
                    [
                    Step1 : Click on the User dropdown
                    Step2 : Click on the logout option in the User dropdown
                    ]
}
"""
    ],
    ids = ["id1","id2"],
)

Add of existing embedding ID: id1
Add of existing embedding ID: id2
Insert of existing embedding ID: id1
Insert of existing embedding ID: id2


In [31]:
all_docs = chroma_collection.get()
all_docs

{'ids': ['id1', 'id2'],
 'embeddings': None,
 'metadatas': [None, None],
 'documents': ['\n{\n    user_instruction : "What are steps to login",\n    System_output : \n                    [\n                    Step1 : Click on the Username inputbox\n                    Step2 : Enter the Username in the Username inputbox\n                    Step3 : click on the password inputbox\n                    Step4 : Enter the password in the password inputbox\n                    Step5 : Click Login Button\n                    ]\n}\n',
  '\n{\n    user_instruction : "What are steps to logout",\n    System_output : \n                    [\n                    Step1 : Click on the User dropdown\n                    Step2 : Click on the logout option in the User dropdown\n                    ]\n}\n'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [24]:
document = chroma_collection.get(ids=['id2'])
document

{'ids': ['id2'],
 'embeddings': None,
 'metadatas': [None],
 'documents': ['\n{\n    user_instruction : "What are steps to logout",\n    System_output : \n                    [\n                    Step1 : Click on the User dropdown\n                    Step2 : Click on the logout option in the User dropdown\n                    ]\n}\n'],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents']}

In [29]:
chroma_collection.query(
    query_texts = ["Query is about dashboard"],
    n_results =2
)

{'ids': [['id2', 'id1']],
 'distances': [[1.7866888046264648, 1.8796885013580322]],
 'metadatas': [[None, None]],
 'embeddings': None,
 'documents': [['\n{\n    user_instruction : "What are steps to logout",\n    System_output : \n                    [\n                    Step1 : Click on the User dropdown\n                    Step2 : Click on the logout option in the User dropdown\n                    ]\n}\n',
   '\n{\n    user_instruction : "What are steps to login",\n    System_output : \n                    [\n                    Step1 : Click on the Username inputbox\n                    Step2 : Enter the Username in the Username inputbox\n                    Step3 : click on the password inputbox\n                    Step4 : Enter the password in the password inputbox\n                    Step5 : Click Login Button\n                    ]\n}\n']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

#### Integrating With Retrival Chain

In [35]:
import langchain
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain

In [47]:
from langchain_community.chat_models import ChatOllama


llm = ChatOllama(model="llama3:latest",temperature=0,)

In [46]:
llm.invoke("tell me a joke")

AIMessage(content="Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(wait for it...)\n\nBecause it was two-tired!\n\nHope that made you smile! Do you want to hear another one?", additional_kwargs={}, response_metadata={'model': 'llama3:latest', 'created_at': '2024-10-07T21:19:22.9244282Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 26478671000, 'load_duration': 10918951800, 'prompt_eval_count': 14, 'prompt_eval_duration': 2721848000, 'eval_count': 40, 'eval_duration': 12832372000}, id='run-45a1c8fb-5b4a-4287-9cf7-49027f59f99d-0')

In [51]:
from langchain_chroma import Chroma
vector_store = Chroma(
    collection_name="orangehrm",

    embedding_function=embed,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

In [ ]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retreiver = vector_store)